<a href="https://colab.research.google.com/github/amontenegrot/COVID19/blob/main/COVID_19_%7C_Plotly.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Importar datos

Datos oficiales:
[Mundo](https://ourworldindata.org/coronavirus-source-data)

[Colombia](https://www.ins.gov.co/Noticias/Paginas/Coronavirus.aspx)

[Bogotá](http://saludata.saludcapital.gov.co/osb/index.php/datos-de-salud/enfermedades-trasmisibles/covid19/?unapproved=410&moderation-hash=b01afdf1c01305df1f6b8a6a02826da5#comment-410)

---
[Genética UNAL](http://www.genetica.unal.edu.co/gibbs/index.php?id=covid19-en-colombia)


https://towardsdatascience.com/top-3-pandas-functions-for-beginners-36373ee47075

https://towardsdatascience.com/3-pandas-functions-you-should-be-using-more-often-696b41e8d7ca

https://towardsdatascience.com/top-3-pandas-functions-you-dont-know-about-probably-5ae9e1c964c8

##Librerias

Lectura de datos y ajustes de visualización

In [ ]:
#%load_ext google.colab.data_table #Visualizar el DataFrame de forma más eficiente en la pantalla

In [ ]:
!pip install cufflinks --upgrade
!pip install pycountry

In [ ]:
import pandas as pd
import numpy as np
import cufflinks as cf
import pycountry

import plotly.express as px
import plotly.graph_objects as go
import plotly.offline as py
from plotly.subplots import make_subplots
from pylab import *
from scipy.optimize import curve_fit
from scipy.integrate import odeint

dfWorld = pd.read_csv('https://covid.ourworldindata.org/data/ecdc/total_cases.csv')
dfMuertes = pd.read_csv('https://covid.ourworldindata.org/data/ecdc/total_deaths.csv')
#dfCol = pd.read_csv('COVID19.csv')
#dfBog = pd.read_csv('COVID19BOG')

##Funciones

In [ ]:
#Función que retorna el código alpha 3 para el país
def get_alph_3 (location):
  try:
    return pycountry.countries.get(name=location).alpha_3
  except:
    return None

In [ ]:
#Función que retorna el código alpha 3 para los departamentos y distritos de Colombia
def get_alph_3_dept (location):
  try:
    return pycountry.subdivisions.get(name=location, country_code='COL')
  except:
    return None

In [ ]:
color1 = 'rgb(34, 124, 114)'
color2 = 'rgb(15, 103, 94)'
color3 = 'rgb(1, 82, 73)'
color4 = 'rgb(0, 59, 53)'
color5 = 'rgb(0, 34, 30)'
color6 = 'rgb(0, 30, 27)'
color7 = 'rgb(55, 164, 117)'
color8 = 'rgb(27, 148, 96)'
color9 = 'rgb(184, 237, 214)'
color10 = 'rgb(131, 214, 178)'

#Mundo

Preprocesamiento de los datos

In [ ]:
#pd.set_option('max_rows', 200) #Quitar los "puntos" de la visualización al imprimir
dfWorld.at[0,'Colombia'] = 0 #Agregar un valor determinado en el DataFrame
dfWorld = dfWorld.fillna(method='ffill') #Rellenar los NAN con el valor anterior en la columna

print(dfWorld.head())
#print(dfWorld['Colombia'].head())

         date  World  Afghanistan  ...  Yemen  Zambia  Zimbabwe
0  2019-12-31     27          0.0  ...    NaN     NaN       NaN
1  2020-01-01     27          0.0  ...    NaN     NaN       NaN
2  2020-01-02     27          0.0  ...    NaN     NaN       NaN
3  2020-01-03     44          0.0  ...    NaN     NaN       NaN
4  2020-01-04     44          0.0  ...    NaN     NaN       NaN

[5 rows x 212 columns]


Procesamiento de los datos

In [ ]:
dfWorld2 = dfWorld.T
dfWorld2 = dfWorld2.drop('date') #Eliminar la columna 'date'
columna = list(dfWorld2.columns) #Crear lista con los nombres de las columnas del DataFrame
columna = columna[-1] #Retornar el último valor de la lista
dfWorld2 = dfWorld2.reset_index() #Reiniciar el índice dada la transposición realizada
dfWorld2 = dfWorld2.rename({'index':'Pais', int(columna):'Total'}, axis=1) #Renombrar columnas

dfWorld2.head()

,Pais,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,...,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,Total
0,World,27,27,27,44,44,59,59,59,59,59,59,59,59,60,60,61,61,66,83,219,239,392,534,631,897,1350,2023,2820,4587,6067,7823,9826,11946,14554,17372,20615,24522,28273,31491,...,4962372,5070115,5176659,5277588,5371597,5460747,5556702,5659329,5778036,5901001,6028732,6136553,6236956,6349555,6476329,6604077,6737602,6863368,6979585,7085464,7211113,7344559,7482088,7626532,7760557,7882634,8001403,8142750,8319213,8457907,8635147,8797066,8926862,9063774,9229573,9400810,9582321,9771518,9952808,10112754
1,Afghanistan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,8145,8676,9216,9998,10582,11173,11831,12456,13036,13659,14525,15205,15750,16509,17267,18054,18969,19551,20342,20917,21459,22143,22890,23546,24102,24766,25527,26310,26874,27532,27878,28424,28833,29143,29481,29715,30175,30451,30616,30967
2,Albania,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,964,969,981,989,998,1004,1029,1050,1076,1099,1122,1137,1143,1164,1184,1197,1212,1232,1246,1263,1299,1341,1385,1416,1464,1521,1590,1672,1722,1788,1838,1891,1927,1995,2047,2114,2192,2269,2330,2402
3,Algeria,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,7542,7728,7918,8113,8306,8503,8697,8857,8997,9134,9267,9394,9513,9626,9626,9831,9935,10050,10154,10265,10382,10484,10589,10698,10810,10919,11031,11147,11268,11385,11504,11631,11771,11920,12076,12248,12445,12685,12968,13273
4,Andorra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,762,762,762,762,763,763,763,763,763,764,764,764,765,844,851,852,852,852,852,852,852,852,852,853,853,853,853,854,854,855,855,855,855,855,855,855,855,855,855,855


Total de casos según el pais

In [ ]:
fig = px.line(dfWorld2, x='Pais', y='Total')
fig.show()

#fig.write_html('CasosMundo.html')

Comparación entre paises

In [ ]:
cols_plot = ['United States', 'China', 'Italy', 'Venezuela', 'Ecuador', 'Colombia'] #Agregar o remover paises (según su nombre en la columna) al gusto
fig = make_subplots(rows=len(cols_plot), cols=1, start_cell="bottom-left")
i = 1
j = 0
for col in cols_plot:
 trace=go.Scatter(x=dfWorld[col], y=dfWorld['date'], mode='lines', name=cols_plot[j])
 fig.append_trace(trace, i, 1)
 i = i+1
 j = j+1
fig.show()

fig.write_html('ContrastePaises.html')

Búsqueda de paises con n número de casos

In [ ]:
dfWorld3 = dfWorld2[(dfWorld2['Total']>=2500)&(dfWorld2['Total']<=3000)] #Paises con un rango entre m y n casos confirmados

dfWorld3 = dfWorld3.T #Transponer DataFrame
dfWorld3 = dfWorld3.reset_index() #Reiniciar index

nombres = dfWorld3.iloc[0] #Llamar nombres del índice 0
dfWorld3 = dfWorld3[1:] #Iniciar el DataFrame desde la fila 1 y no desde la fila 0
dfWorld3.columns=nombres #Asignar los nombres a las columnas
dfWorld3 = dfWorld3.drop(['Pais'], axis=1) #Eliminar el valor 'Pais' de la lista de nombres

paises = list(dfWorld3) #Extracción de los nombres de paises que cumplen la condición

print(paises)

['Croatia', 'Somalia']


Gráfica de países que cumplen la condición

In [ ]:
py.iplot([{
   'x': dfWorld3.index,
   'y': dfWorld3[col],
   'name': col
}  for col in dfWorld3.columns], filename='cufflinks/multiple-lines-on-same-chart')

fig.write_html('NumeroCasos.html')

Mapas

In [ ]:
dfWorld2

,Pais,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,...,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,Total
0,World,27,27,27,44,44,59,59,59,59,59,59,59,59,60,60,61,61,66,83,219,239,392,534,631,897,1350,2023,2820,4587,6067,7823,9826,11946,14554,17372,20615,24522,28273,31491,...,4962372,5070115,5176659,5277588,5371597,5460747,5556702,5659329,5778036,5901001,6028732,6136553,6236956,6349555,6476329,6604077,6737602,6863368,6979585,7085464,7211113,7344559,7482088,7626532,7760557,7882634,8001403,8142750,8319213,8457907,8635147,8797066,8926862,9063774,9229573,9400810,9582321,9771518,9952808,10112754
1,Afghanistan,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,8145,8676,9216,9998,10582,11173,11831,12456,13036,13659,14525,15205,15750,16509,17267,18054,18969,19551,20342,20917,21459,22143,22890,23546,24102,24766,25527,26310,26874,27532,27878,28424,28833,29143,29481,29715,30175,30451,30616,30967
2,Albania,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,964,969,981,989,998,1004,1029,1050,1076,1099,1122,1137,1143,1164,1184,1197,1212,1232,1246,1263,1299,1341,1385,1416,1464,1521,1590,1672,1722,1788,1838,1891,1927,1995,2047,2114,2192,2269,2330,2402
3,Algeria,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,7542,7728,7918,8113,8306,8503,8697,8857,8997,9134,9267,9394,9513,9626,9626,9831,9935,10050,10154,10265,10382,10484,10589,10698,10810,10919,11031,11147,11268,11385,11504,11631,11771,11920,12076,12248,12445,12685,12968,13273
4,Andorra,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,762,762,762,762,763,763,763,763,763,764,764,764,765,844,851,852,852,852,852,852,852,852,852,853,853,853,853,854,854,855,855,855,855,855,855,855,855,855,855,855
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
206,Vietnam,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,2,2,2,2,2,2,2,5,5,7,8,9,10,10,12,...,324,324,324,324,325,326,327,327,327,327,327,328,328,328,328,328,329,329,331,332,332,332,332,333,334,334,334,335,335,342,349,349,349,349,349,352,352,353,354,355
207,Western Sahara,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,6,6,6,6,6,6,6,6,6,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,23,25,25,25,25,25,25,25,25,208,208
208,Yemen,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,180,193,205,212,222,237,249,255,278,287,310,323,354,399,403,453,469,473,486,496,524,560,591,632,705,728,844,889,902,906,919,923,941,967,992,1015,1076,1089,1103,1118
209,Zambia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,832,866,920,920,920,920,920,1057,1057,1057,1057,1057,1089,1089,1089,1111,1111,1154,1154,1200,1200,1252,1252,1321,1357,1358,1382,1405,1412,1416,1430,1430,1430,1430,1477,1489,1497,1531,1531,1557


##Mapas

In [ ]:
dfWorld2['Codigo'] = dfWorld2['Pais'].apply(lambda x: get_alph_3(x)) #Crear una columna con el código alpha 3 de cada país

print(dfWorld2.head())

          Pais    0    1    2    3  ...      178      179      180     Total Codigo
0        World   27   27   27   44  ...  9582321  9771518  9952808  10112754   None
1  Afghanistan    0    0    0    0  ...    30175    30451    30616     30967    AFG
2      Albania  NaN  NaN  NaN  NaN  ...     2192     2269     2330      2402    ALB
3      Algeria    0    0    0    0  ...    12445    12685    12968     13273    DZA
4      Andorra  NaN  NaN  NaN  NaN  ...      855      855      855       855    AND

[5 rows x 184 columns]


In [ ]:
fig = px.choropleth(dfWorld2, 
                    locations='Codigo',
                    color='Total',
                    hover_name='Pais',
                    color_continuous_scale=px.colors.sequential.Plasma)

fig.show()

fig.write_html('CasosMundial.html')

Series de tiempo

In [ ]:
dfWorld4 = dfWorld.fillna(method='ffill') #Rellenar los NAN con el valor anterior en la columna
dfWorld4['date'] = pd.to_datetime(dfWorld4['date']) #Conversión a formato de fecha a la columna 'date'
dfWorld4 = dfWorld4.set_index('date') #Se selecciona la columna 'date' como índice del DataFrame

In [ ]:
resumen = pd.DataFrame() #Creación de un nuevo DataFrame
resumen['PSemanal'] = dfWorld4['Colombia'].resample('W-MON').mean().round(2) #Creación de nueva columna con un promedio de la suma de casos por semana (iniciando el lunes)
print(resumen)

            PSemanal
date                
2020-01-06      0.00
2020-01-13      0.00
2020-01-20      0.00
2020-01-27      0.00
2020-02-03      0.00
2020-02-10      0.00
2020-02-17      0.00
2020-02-24      0.00
2020-03-02      0.00
2020-03-09      0.43
2020-03-16     17.00
2020-03-23    136.43
2020-03-30    499.14
2020-04-06   1155.43
2020-04-13   2227.71
2020-04-20   3288.71
2020-04-27   4635.00
2020-05-04   6603.29
2020-05-11   9515.71
2020-05-18  13593.43
2020-05-25  18532.86
2020-06-01  25537.29
2020-06-08  34956.86
2020-06-15  45243.43
2020-06-22  60402.57
2020-06-29  81038.43


In [ ]:
resumen['PSemanalMin'] = dfWorld4['Colombia'].resample('W-MON').min() #Creación de nueva columna con el menor número de casos promedio por semana (iniciando el lunes)
print(resumen)

            PSemanal  PSemanalMin
date                             
2020-01-06      0.00          0.0
2020-01-13      0.00          0.0
2020-01-20      0.00          0.0
2020-01-27      0.00          0.0
2020-02-03      0.00          0.0
2020-02-10      0.00          0.0
2020-02-17      0.00          0.0
2020-02-24      0.00          0.0
2020-03-02      0.00          0.0
2020-03-09      0.43          0.0
2020-03-16     17.00          3.0
2020-03-23    136.43         57.0
2020-03-30    499.14        306.0
2020-04-06   1155.43        798.0
2020-04-13   2227.71       1579.0
2020-04-20   3288.71       2852.0
2020-04-27   4635.00       3977.0
2020-05-04   6603.29       5597.0
2020-05-11   9515.71       7973.0
2020-05-18  13593.43      11613.0
2020-05-25  18532.86      16295.0
2020-06-01  25537.29      21981.0
2020-06-08  34956.86      30493.0
2020-06-15  45243.43      40719.0
2020-06-22  60402.57      53063.0
2020-06-29  81038.43      71183.0


In [ ]:
resumen['PSemanalMax'] = dfWorld4['Colombia'].resample('W-MON').max() #Creación de nueva columna con el mayor número de casos promedio por semana (iniciando el lunes)
print(resumen)

            PSemanal  PSemanalMin  PSemanalMax
date                                          
2020-01-06      0.00          0.0          0.0
2020-01-13      0.00          0.0          0.0
2020-01-20      0.00          0.0          0.0
2020-01-27      0.00          0.0          0.0
2020-02-03      0.00          0.0          0.0
2020-02-10      0.00          0.0          0.0
2020-02-17      0.00          0.0          0.0
2020-02-24      0.00          0.0          0.0
2020-03-02      0.00          0.0          0.0
2020-03-09      0.43          0.0          1.0
2020-03-16     17.00          3.0         45.0
2020-03-23    136.43         57.0        235.0
2020-03-30    499.14        306.0        702.0
2020-04-06   1155.43        798.0       1485.0
2020-04-13   2227.71       1579.0       2776.0
2020-04-20   3288.71       2852.0       3792.0
2020-04-27   4635.00       3977.0       5379.0
2020-05-04   6603.29       5597.0       7668.0
2020-05-11   9515.71       7973.0      11063.0
2020-05-18  1

In [ ]:
resumen['AcumFinSemana'] = dfWorld4['Colombia'].resample('W-MON').last() #Creación de nueva columna con el mayor número de casos promedio por semana (iniciando el lunes)
print(resumen)

            PSemanal  PSemanalMin  PSemanalMax  AcumFinSemana
date                                                         
2020-01-06      0.00          0.0          0.0            0.0
2020-01-13      0.00          0.0          0.0            0.0
2020-01-20      0.00          0.0          0.0            0.0
2020-01-27      0.00          0.0          0.0            0.0
2020-02-03      0.00          0.0          0.0            0.0
2020-02-10      0.00          0.0          0.0            0.0
2020-02-17      0.00          0.0          0.0            0.0
2020-02-24      0.00          0.0          0.0            0.0
2020-03-02      0.00          0.0          0.0            0.0
2020-03-09      0.43          0.0          1.0            1.0
2020-03-16     17.00          3.0         45.0           45.0
2020-03-23    136.43         57.0        235.0          235.0
2020-03-30    499.14        306.0        702.0          702.0
2020-04-06   1155.43        798.0       1485.0         1485.0
2020-04-

Contraste entre países por mes

In [ ]:
dfWorld4 = dfWorld4.filter(items=['Colombia', 'United States', 'Spain', 'Italy', 'Venezuela', 'Ecuador'])

In [ ]:
pd.set_option('display.max_columns', None) #Configuración de columnas sin cortes
pd.set_option('display.expand_frame_repr', False) #Configuración de columnas sin cortes

resumen2 = pd.DataFrame()
resumen2 = dfWorld4.resample('M').last() #Configuración del nuevo DataFrame para hacer un resumen por mes
print(resumen2)

            Colombia  United States     Spain   Italy  Venezuela  Ecuador
date                                                                     
2019-12-31       0.0              0       0.0       0        NaN      0.0
2020-01-31       0.0              6       0.0       3        NaN      0.0
2020-02-29       0.0             66      54.0     888        NaN      0.0
2020-03-31     798.0         164620  104267.0  101739      135.0   1966.0
2020-04-30    6211.0        1039909  215183.0  203591      331.0  24675.0
2020-05-31   28236.0        1770384  239429.0  232664     1459.0  38571.0
2020-06-30   91769.0        2548996  248770.0  240310     5297.0  55255.0


In [ ]:
diferencias = dfWorld4.diff()
diferencias = diferencias.abs()
print(diferencias)

            Colombia  United States  Spain  Italy  Venezuela  Ecuador
date                                                                 
2019-12-31       NaN            NaN    NaN    NaN        NaN      NaN
2020-01-01       0.0            0.0    0.0    0.0        NaN      0.0
2020-01-02       0.0            0.0    0.0    0.0        NaN      0.0
2020-01-03       0.0            0.0    0.0    0.0        NaN      0.0
2020-01-04       0.0            0.0    0.0    0.0        NaN      0.0
...              ...            ...    ...    ...        ...      ...
2020-06-25    3541.0        34339.0  400.0  577.0      179.0    691.0
2020-06-26    3486.0        40949.0  419.0  296.0      198.0    822.0
2020-06-27    3843.0        45527.0  564.0  255.0      216.0    700.0
2020-06-28    4149.0        42486.0  301.0  175.0      351.0    718.0
2020-06-29    3178.0        38673.0    0.0  174.0      167.0    681.0

[182 rows x 6 columns]


In [ ]:
cols_plot = ['Colombia', 'United States', 'Spain', 'Italy', 'Venezuela', 'Ecuador']
fig = make_subplots(rows=len(cols_plot), cols=1, start_cell="bottom-left")
i=1
j=0
for col in cols_plot:
 trace=go.Scatter(x=diferencias.index, y=diferencias[col],mode='lines',name=cols_plot[j])
 fig.append_trace(trace,i,1)
 i=i+1
 j=j+1
fig.show()

fig.write_html('MitigacionCasos.html')

#Colombia

Preprocesmiento de los datos

In [ ]:
#print(dfCol.shape) #Tamaño del DataFrame
#print(df.dtypes) #Tipo de datos para cada columna

dfCol = dfCol.drop('ID de caso', axis=1) #Eliminar la columna 'ID de caso'
#dfCol = dfCol.reset_index() #Reiniciar el Index dada la transposición realizada
dfCol = dfCol.rename({'Fecha de diagnóstico':'Fecha', 
                      'Ciudad de ubicación':'Ciudad', 
                      'Departamento o Distrito':'Departamento',
                      'Atención**':'Estado',
                      'Tipo*':'Tipo',
                      'País de procedencia':'Procedencia'}, axis=1) #Renombrar columnas

dfCol['Casos'] = 1
dfCol.head()

NameError: ignored

In [ ]:
sexo = dfCol['Estado'].value_counts() #Resumen de valores en la columna 'Estado'

hombresT = pd.DataFrame()
mujeresT = pd.DataFrame()

hombres = dfCol[(dfCol['Sexo'] == 'M')&(dfCol['Estado'] != 'Recuperado')&(dfCol['Estado'] != 'Fallecido')] #Búsqueda de hombres sin contar fallecidos o con segunda prueba negativa
hombresT['Fecha'] = hombres['Fecha']
hombresT['Casos'] = hombres['Casos']
hombres = hombres['Sexo'] #Asignar la columna 'Sexo' al DataFrame
hombres = hombres.count() #Contar valores del DataFrame

mujeres = dfCol[(dfCol['Sexo'] == 'F')&(dfCol['Estado'] != 'Recuperado')&(dfCol['Estado'] != 'Fallecido')] #Búsqueda de mujeres sin contar fallecidos o con segunda prueba negativa
mujeresT['Fecha'] = mujeres['Fecha']
mujeresT['Casos'] = mujeres['Casos']
mujeres = mujeres['Sexo'] #Asignar la columna 'Sexo' al DataFrame
mujeres = mujeres.count() #Contar valores del DataFrame

#print(sexo)
#print(hombres.count())

In [ ]:
universidades=['U. de los Andes','U. Distrital Francisco<br> José de Caldas','U. Externado de Colombia','U. Konrad Lorenz','U. Nacional de Colombia','U. Pedagógica Nacional']
xH=[0,2,4,0,6,0]
xM=[1,3,5,1,3,2]


fig = go.Figure()
fig.add_trace(go.Bar(
    y=universidades,
    x=xH,
    text=xH,
    textposition='auto',
    name='Hombres',
    marker_color=color3,
    orientation='h'
))
fig.add_trace(go.Bar(
    y=universidades,
    x=xM,
    text=xM,
    textposition='auto',
    name='Mujeres',
    marker_color=color10,
    orientation='h'
))

fig.update_layout(
    xaxis_title="Número de casos confirmados",
    yaxis_title="Departamento",
    autosize=False,
    width=1000,
    height=350,
    barmode='stack',
    margin=dict(l=0.5, r=0.5, t=3, b=3)
    )
fig.show()

Casos de hombres contagiados (sin contar fallecidos o con segunda prueba negativa)

In [ ]:
fig = go.Figure(go.Indicator(
    mode = "number+delta",
    value = hombres,
    delta = {"reference": 513, "valueformat": ".0f"},
    title = {"text": "Hombres"},
    domain = {'y': [0, 1], 'x': [0.25, 0.75]}))

fig.add_trace(go.Scatter(
    y = hombresT['Fecha']))

fig.update_layout(xaxis = {'range': [0, hombresT['Casos']]})
fig.show()

Casos de mujeres contagiados (sin contar fallecidos o con segunda prueba negativa)

In [ ]:
fig = go.Figure(go.Indicator(
    mode = "number+delta",
    value = mujeres,
    delta = {"reference": 513, "valueformat": ".0f"},
    title = {"text": "Mujeres"},
    domain = {'y': [0, 1], 'x': [0.25, 0.75]}))

fig.add_trace(go.Scatter(
    y = mujeresT['Fecha']))

fig.update_layout(xaxis = {'range': [0, mujeresT['Casos']]})
fig.show()

##Departamentos

In [ ]:
dfColDept = dfCol.groupby(['Departamento'])[['Casos']].sum()
dfColDept = dfColDept.reset_index() #Reiniciar el Index

DeptMaximo = dfColDept.loc[dfColDept['Casos'].idxmax()].at['Departamento'] #Valor en 'Departamento' según  el índice con valor máximo en 'Casos'
CasoMaximoDept = dfColDept['Casos'].max() #Valor máximo en la columna 'Casos'

print(DeptMaximo, 'es el departamento/distrito con mayor número de casos confirmados de COVID-19:', CasoMaximoDept,'\n')

print('La distribución de casos confirmados de COVID-19 por departamento/distrito es la siguiente:\n')
print(dfColDept)

Mapa

In [ ]:
dfCol['CodigoDept'] =dfCol['Departamento'].apply(lambda x: get_alph_3_dept(x)) #Crear una columna con el código alpha 3 de cada departamento

print(dfCol.head())

##Ciudades

In [ ]:
dfColCiu = dfCol.groupby(['Ciudad'])[['Casos']].sum()
dfColCiu = dfColCiu.reset_index() #Reiniciar el Index

CiuMaxima = dfColCiu.loc[dfColCiu['Casos'].idxmax()].at['Ciudad'] #Valor en 'Ciudad' según  el índice con valor máximo en 'Casos'
CasoMaximoCiu = dfColCiu['Casos'].max() #Valor máximo en la columna 'Casos'

print(CiuMaxima, 'es la ciudad con mayor número de casos confirmados de COVID-19:', CasoMaximoCiu,'\n')

print('La distribución de casos confirmados de COVID-19 por ciudades es la siguiente:\n')
print(dfColCiu)

##Ecuación exponencial

In [ ]:
primerCasoCol = dfWorld.loc[(dfWorld['Colombia']>=1).idxmax(), 'date'] #Saber la fecha de el(los) primer(os) caso(s) reportados en el pais
print(primerCasoCol)

In [ ]:
dfWorld['date'] = pd.to_datetime(dfWorld['date']) #Cambiar los datos de la columna al tipo 'fecha'

condicion = dfWorld['date'] >= pd.Timestamp(primerCasoCol) #Verificar que la fecha sea mayor a la del primer caso
casosCol = dfWorld['Colombia'].loc[condicion,] #Verificar elcumplimiento de la condición en el pais
numeroCasos = len(casosCol) #número de casos desde el primer día hasta el último
print(numeroCasos)
print(casosCol)

In [ ]:
fecha = np.arange(1,numeroCasos+1,1) #Números ascendentes de uno en uno
print(len(fecha) == numeroCasos) #Verificar que coincidan el número de días con el tamaño de la matriz

Gráfica

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=casosCol, x=fecha,mode='lines',name='MX'))
fig.show()

#fig.write_html('Exponencial1.html')

Implementación de la librería Scipy

In [ ]:
x=fecha
y=casosCol
def func(x, c, k):
   return (c*np.exp(k*x))
 
popt, pcov = curve_fit(func, x, y)
print(popt)

print(func(x,*popt))#'*popt' envía los valores de la variable 'popt' por separado, sin necesidad de usar 'popt[0]' y 'popt[1]'

In [ ]:
fig = go.Figure()
fig.add_trace(go.Scatter(y=func(x,*popt), x=fecha,mode='lines',name='Colombia (Ajuste)'))
fig.add_trace(go.Scatter(y=casosCol, x=fecha,mode='lines',name='Colombia'))
fig.show()

#fig.write_html('Exponencial2.html')

In [ ]:
print(pycountry.countries.search_fuzzy('Colombia'))

print(pycountry.subdivisions.get(country_code='COL'))

#Modelo SIR

Modelo estadístico que pretende describir la propagación de determinada enfermedad en una población específica. En ese sentido, el modelo contempla tres clasificaciones en relación al tiempo:

$t$: tiempo \\
$N$: población

$S(t)$: personas suceptibles a enfermarse (no han sido contagiadas) \\
$I(t)$: personas infectadas \\
$R(t)$: personas recuperadas (inmunes)

$β$: tasa de transmisión \\
$γ$: tasa de recuperación

##Parametros
$N$: 48.258.494 \\
$β$: 2.2 \\
$γ$: 1/5.2

##Condiciones iniciales

$R(0)$: 0 \\
$I(1)$: 1

In [ ]:
N =  48258494 #Población total (Colombia)

I0 = 1 #Valores iniciales de personas Infectadas (I0)
R0 = 0 #Valores iniciales de personas que se recuperaron (R0)

#El resto de la población, S0 son las personas que están sujetos a infección inicialmente.
S0 = N - I0 - R0

# Tasa de transmisión, tasa de recuperación en 1/días
beta = 2.2
gamma = 1/5.2

In [ ]:
#Puntos en la gráfica (en días)
t = np.linspace(0, 365, 365)

In [ ]:
#Las ecuaciones diferenciales del modelo SIR
def deriv(y, t, N, beta, gamma):
   S, I, R = y
   dSdt = -beta * S * I / N
   dIdt = beta * S * I / N - gamma * I
   dRdt = gamma * I
   return dSdt, dIdt, dRdt

In [ ]:
#Vector de las condiciones iniciales
y0 = S0, I0, R0

#Resolver el sistema de ecuaciones diferenciales, en la secuencia de días que ya definimos
ret = odeint(deriv, y0, t, args=(N, beta, gamma))
S, I, R = ret.T

In [ ]:
#Plot the data on three separate curves for S(t), I(t) and R(t)
fig = go.Figure()
fig.add_trace(go.Scatter(x=t, y=I,mode='lines',name='Infected'))
fig.show()

In [ ]:
valores = [(2.2,1/2.3), (5,1/7), (1,1/3)]

def modelo(beta, gamma):
  N =  48258494 #Población total (Colombia)
  I0 = 1 #Valores iniciales de personas Infectadas (I0)
  R0 = 0 #Valores iniciales de personas que se recuperaron (R0)

  #El resto de la población, S0 son las personas que están sujetos a infección inicialmente.
  S0 = N - I0 - R0

  #Puntos en la gráfica (en días)
  t = np.linspace(0, 365, 365)

  #Las ecuaciones diferenciales del modelo SIR
  def deriv(y, t, N, beta, gamma):
    S, I, R = y
    dSdt = -beta * S * I / N
    dIdt = beta * S * I / N - gamma * I
    dRdt = gamma * I
    return dSdt, dIdt, dRdt

  #Vector de las condiciones iniciales
  y0 = S0, I0, R0

  #Resolver el sistema de ecuaciones diferenciales, en la secuencia de días que ya definimos
  ret = odeint(deriv, y0, t, args=(N, beta, gamma))
  S, I, R = ret.T
  return(I)

#Para la parte de las gráficas:
py.iplot([{
   'x': t,
   'y': modelo(*valor),
   'name': str(valor),
}  for valor in valores], filename='cufflinks/multiple-lines-on-same-chart')  

####Última línea